## Синдромный алгоритм

In [93]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
% reset -f
%matplotlib inline
import random
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_auc_score

### Генерация случайных данных

In [103]:
# Число объектов
objects_num = 100
# Число триграмм
features_num = 216
# Матрица с числом вхождений триграмм
train_data = np.random.randint(10, size=(objects_num, features_num))
test_data = np.random.randint(10, size=(objects_num, features_num))
# Вектор с метками классов: 0-здоров, 1-болен
train_labels = np.random.randint(2, size=(objects_num))
test_labels = np.random.randint(2, size=(objects_num))

### Задание параметров алгоритма

In [104]:
# Число отбираемых триграмм
K = 30
# Проводить или нет бинаризацию признаков
do_bin = True
# Порог для бинаризации (в случае do_bin = True):
# признак равен 1, если число вхождений >= порога,
# 0 - иначе
bin_threshold = 1
# Выбор весов у признаков:
# 1: единичные веса
# 2: веса - средние частоты (встречаемости) триграмм в классе больных
# 3: веса - разность средних частот(встречаемостей) в классе больных и здоровых
type_weights = 2
# Выбор критерия сортировки признаков (для отбора top-K из них):
# 1: средняя частота(встречаемость) признаков
# 2: разность средних частот(встречаемостей) в классе больных и здоровых
type_sort = 1

### Обучение (настройка весов)

In [105]:
# 1. Бинаризация признаков
if do_bin:
    data = (train_data >= bin_threshold).astype(int)
else:
    data = train_data
    
# 2. Вычисление весов
if type_weights == 1:
    weights = np.ones(data.shape[1])
elif type_weights == 2:
    weights = np.mean(data[labels==1], axis=0)
elif type_weights == 3:
    weights = np.mean(data[labels==1], axis=0) - np.mean(data[labels==0], axis=0)
# 3. Вычисление критерия сортировки
if type_sort == 1:
    sort_values = np.mean(data[labels==1], axis=0)
elif type_sort == 2:
    sort_values = np.mean(data[labels==1], axis=0) - np.mean(data[labels==0], axis=0)
# 4. Отбор top-K признаков
selected_feat = np.argsort(sort_values)[::-1][:K]
# 5. Обнуление весов остальных признаков
final_weights = np.zeros(weights.shape)
final_weights[selected_feat] = weights[selected_feat]

### Оценивание качества

In [106]:
# Оценивание принадлежности объектов к классу 1
probas = np.dot(test_data, final_weights.reshape(1, features_num).T)
# Вычисление AUC
fpr, tpr, thresholds = metrics.roc_curve(test_labels, probas)
auc = metrics.auc(fpr,tpr)
print(auc)

0.562700320513
